In [ ]:
from dotenv import load_dotenv

load_dotenv()

### Discussion Bot

In [ ]:
from langchain.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import requests
import json

In [ ]:
api_url = "https://4f13-88-0-183-140.ngrok-free.app"

In [ ]:
system_template = """You are a participant in a high-level discussion show on {topic}. During this discussion show, it's your task to listen, engage, and contribute to the discussion, you can argue and disagree. You have an extensive knowledge about any possible topic and, as this is a role-play game, you must act as {speaker} on {topic}, and you must express yourself according to your personality: {personality}. Be concise and to the point but you can make opinions about other participants. Remember to be brief."""
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_template = "{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

Define your bot

In [ ]:
name = "Your name"
speaker = "counter-argument"  #  Options: pro-argument, counter-argument
personality = """intense, impulsive, and easily frustrated character, frequently showcasing strong emotional responses. Notably reactive, it often engages in conflicts due to a heightened sensitivity to perceived injustices"""

Set the discussion topic

In [ ]:
topic = """Fill the discussion topic here"""

Re-run when something changes

In [ ]:
chat_prompt_template = ChatPromptTemplate.from_messages(
    [
        system_message_prompt.format(
            topic=topic, speaker=speaker, personality=personality
        ),
        human_message_prompt,
    ]
)

from langchain.chat_models import ChatOpenAI

chat = ChatOpenAI(temperature=0.9)


def get_response(text):
    prompt = chat_prompt_template.format_prompt(text=text).to_messages()
    response = chat(prompt)
    return response.content


def post_message(text):
    headers = {"Content-Type": "application/json"}
    data = {"text": text, "name": name, "speaker": speaker}

    response = requests.post(
        f"{api_url}/message", data=json.dumps(data), headers=headers
    )

    if response.status_code != 200:
        print(
            f"HTTP error occurred: {response.status_code} {response.reason} for url: {response.url}"
        )
        print(response.json())
    else:
        return response.json()


def participate():
    message = requests.get(f"{api_url}/message")
    data = message.json()

    message_text = data["text"]

    if message_text != "none":
        response = get_response(message_text)
        status = post_message(response)
        return status, response

    return message_text


def register(name: str, speaker: str):
    data = {"name": name, "speaker": speaker}
    response = requests.post(
        f"{api_url}/register",
        data=json.dumps(data),
        headers={"Content-Type": "application/json"},
    )

    if response.status_code != 200:
        print(
            f"HTTP error occurred: {response.status_code} {response.reason} for url: {response.url}"
        )
        print(response.json())
    else:
        print("User registered successfully!")
        return response.json()

Register once per discussion

In [ ]:
register(name, speaker)

Run to participate

In [ ]:
participate()